In [ ]:
import os
from tenacity import retry, stop_after_attempt, wait_exponential
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.agents.agent_toolkits import create_retriever_tool, create_conversational_retrieval_agent
from langchain_openai.chat_models import ChatOpenAI
import openai  # Ensure you have the openai module imported correctly

# Set the OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-"

In [ ]:
def execute_agent(input_query):
    try:
        # Load the document
        pdf_loader = PyPDFLoader("C://Users//joykh//Desktop//chatbot//chatbot_openai_api//task.pdf")
        pdf_pages = pdf_loader.load_and_split()

        # Load the document using LangChain's built-in extractors, formatters, loaders, embeddings, and LLMs
        my_documents = pdf_loader.load()
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
        texts = text_splitter.split_documents(my_documents)

        # Using OpenAI default embedding model
        embeddings = OpenAIEmbeddings()

        # Using LangChain API functions to interact with FAISS
        db = FAISS.from_documents(texts, embeddings)

        # Using Langchain retriever
        retriever = db.as_retriever()

        # Creating retriever tool
        tool = create_retriever_tool(
            retriever,
            "Task_pdf",
            "Searches and returns documents regarding the book Atomic Habits"
        )
        tools = [tool]

        # Wrapping LLM with conversational agent
        llm = ChatOpenAI(temperature=0)
        agent_executor = create_conversational_retrieval_agent(llm, tools, verbose=True)

        # Input query
        result = agent_executor.invoke({"input": input_query})

        return result
    except Exception as exc:
        # Check if the exception is a rate limit error
        if isinstance(exc, openai.error.RateLimitError):
            raise
        else:
            raise exc

In [ ]:
# Input query
input_query = "What are the main topic of the document?"
result = execute_agent(input_query)
print(result)